# Running pre-trained models on biomedical texts to evaluate them
## Infrastructure

In [38]:
!pip install transformers
!pip install tqdm
!pip install sentencepiece
!pip install sacremoses
!pip install accelerate
!pip install ipywidgets
!pip install protobuf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


...

DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621
DEPRECATION: Configuring installation scheme with distutils config files is deprecated and will no longer work in the near future. If you are using a Homebrew or Linuxbrew Python, please see discussion at https://github.com/Homebrew/homebrew-core/issues/76621


In [39]:
from abc import abstractmethod

using_gpu = False
device = "gpu" if using_gpu else "cpu"


class TranslationModel:
    def __init__(self, checkpoint_name: str):
        self.checkpoint_name = checkpoint_name

    @abstractmethod
    def translate(self, source: str) -> str:
        """Translates a source text with the model
        :param source: the text to translate
        :return: str - the translation"""
        pass

    def __str__(self):
        return self.checkpoint_name

### Helsinki-NLP/opus-mt-en-es

In [40]:
from transformers import MarianMTModel, MarianTokenizer


class HelsinkiNLPModel(TranslationModel):
    def __init__(self):
        super().__init__("Helsinki-NLP/opus-mt-en-es")
        self.model = MarianMTModel.from_pretrained(self.checkpoint_name)
        self.tokenizer = MarianTokenizer.from_pretrained(self.checkpoint_name)

    def translate(self, source: str) -> str:
        input_ids = self.tokenizer.encode(source, return_tensors="pt")
        translated_tokens = self.model.generate(input_ids, num_beams=4, early_stopping=True)
        translated_text = self.tokenizer.decode(translated_tokens[0], skip_special_tokens=True)
        return translated_text

### facebook/nllb-200-distilled-600M

In [41]:
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM


class NLLBModel(TranslationModel):
    def __init__(self):
        super().__init__("facebook/nllb-200-distilled-600M")
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint_name)
        self.model = AutoModelForSeq2SeqLM.from_pretrained(self.checkpoint_name, device_map="auto")

    def translate(self, source: str) -> str:
        inputs = self.tokenizer(source, return_tensors="pt")
        translated_tokens = self.model.generate(
            **inputs, forced_bos_token_id=self.tokenizer.lang_code_to_id["spa_Latn"], max_length=30
        )
        translated_text = self.tokenizer.batch_decode(translated_tokens, skip_special_tokens=True)[0]
        return translated_text

## Madlad (Google T5)

In [46]:
from transformers import T5Tokenizer, T5ForConditionalGeneration


class MadladModel(TranslationModel):
    def __init__(self):
        super().__init__('jbochi/madlad400-3b-mt')
        self.tokenizer = T5Tokenizer.from_pretrained(self.checkpoint_name)
        self.model = T5ForConditionalGeneration.from_pretrained(self.checkpoint_name, device_map="auto")

    def translate(self, source: str) -> str:
        input_ids = self.tokenizer(f"<2es> {source}", return_tensors="pt").input_ids.to(self.model.device)
        outputs = self.model.generate(input_ids=input_ids)
        translated_text = self.tokenizer.decode(outputs[0], skip_special_tokens=True)
        return translated_text


## Loading and evaluation functions

In [43]:
import json
import os

import pandas as pd
from tqdm import tqdm

from pretrained_models.preliminary_experiments.translation_model import TranslationModel, NLLBModel


def load_sentences(test_dataset: str, num_rows=1000, seed=42) -> pd.DataFrame:
    """Loads a test dataset in .jsonl format into a dataframe and randomly selects n rows
    :param test_dataset: filename of the test dataset
    :param num_rows: number of rows to select
    :param seed: random seed for reproducibility
    :return pd.Dataframe: the parallel corpus as a dataframe"""
    data = []
    with open(test_dataset, "r") as f:
        for line in f:
            data.append(json.loads(line))
    df = pd.DataFrame(data)
    return df.sample(n=num_rows, random_state=seed)


def run_models(mt_models: list[TranslationModel], test_sentences: pd.DataFrame, dataset_name: str):
    """Runs each model on the test sentences. Creates a dataframe for the results with two columns: 'Reference' and 'Actual'
    :param mt_models: the list of translation models to evaluate
    :param test_sentences: a dataframe for a parallel corpus"""

    dataset_without_extension = dataset_name.split(".jsonl")[0]
    folder = f"translations/{dataset_without_extension}/"

    for model in mt_models:
        print(f"\tModel: {model}")
        df = pd.DataFrame(columns=['reference', 'actual'])

        for _, row in tqdm(test_sentences.iterrows()):
            english, spanish = row['en'], row['es']
            translation = model.translate(english)
            df.loc[len(df.index)] = [spanish, translation]

        result_filename = folder + model.checkpoint_name.split("/")[0] + ".csv"
        os.makedirs(os.path.dirname(result_filename), exist_ok=True)
        df.to_csv(result_filename)


def evaluate_on_all_test_data(translation_models: list[TranslationModel], dataset_names: list[str],
                              corpus_directory: str):
    for test_dataset in dataset_names:
        test_path = corpus_directory + test_dataset
        print(f"Test dataset: {test_dataset}")
        test_sentences = load_sentences(test_path, num_rows=5)

        run_models(translation_models, test_sentences, test_dataset)

## Evaluating the models

In [47]:
all_models = [MadladModel()]
# HelsinkiNLPModel(), NLLBModel(), 

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [48]:
test_directory = "/Users/zaki/PycharmProjects/hpo_translation/corpus/train/"
filenames = ["abstracts.jsonl", "khresmoi.jsonl", "medline.jsonl", "scielo.jsonl", "snomed.jsonl"]

evaluate_on_all_test_data(all_models, filenames, test_directory)

Test dataset: abstracts.jsonl
	Model: t5-base


5it [00:21,  4.40s/it]


Test dataset: khresmoi.jsonl
	Model: t5-base


5it [00:26,  5.21s/it]


Test dataset: medline.jsonl
	Model: t5-base


5it [00:57, 11.54s/it]


Test dataset: scielo.jsonl
	Model: t5-base


5it [00:25,  5.17s/it]


Test dataset: snomed.jsonl
	Model: t5-base


5it [00:07,  1.43s/it]
